In [1]:
import pickle
# import optuna
# from optuna.trial import TrialState
from gnn import gnn
import time
import numpy as np
import utils
from utils import *
import torch.nn as nn
import torch
import time
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'
# os.envirment[]
from collections import defaultdict
import argparse
import time
from torch.utils.data import DataLoader          
# from torch.utils.data import DataLoader
from prefetch_generator import BackgroundGenerator

/home/caoduanhua/anaconda3/envs/rdkit-env/lib/python3.6/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/caoduanhua/score_function/GNN/GNN_graphformer_pyg/algos.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [6]:
sample_path = '/home/caoduanhua/score_function/data/general_refineset/refineset_active_pocket_without_h/1a4k_ligand_active_0'
# sample = 
with  open(sample_path,'rb') as f:
    m1,m2 = pickle.load(f)

In [4]:
sample[0].GetBonds()

In [18]:
pocket = (m1,m2)
getEdge(pocket,n1 = len(m1.GetAtoms()),n2 = len(m2.GetAtoms()))

(array([[  0,   1,   1, ..., 402, 404, 402],
        [  1,   0,   3, ..., 403, 402, 404]]),
 tensor([[1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [1, 0, 1, 0, 0],
         ...,
         [2, 0, 1, 0, 0],
         [1, 0, 1, 0, 0],
         [1, 0, 1, 0, 0]]))

In [16]:

def molEdge(mol,adj = None,n1= None,n2=None):
    edges_list = []
    edge_features_list = []
    if len(mol.GetBonds()) > 0: # mol has bonds
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx() 
            j = bond.GetEndAtomIdx()
            edge_feature = bond_to_feature_vector(bond)
            # add edges in both directions
            edges_list.append((i, j))
            edge_features_list.append(edge_feature)
            edges_list.append((j, i))
            edge_features_list.append(edge_feature)
    # add virtual aromatic nodes feature
    # add mol 
    if adj== None:
        return edges_list ,edge_features_list
    else:
        n = len(mol.GetAtom())
        adj -= np.eye(len(adj))
        dm = adj[n:n1,:n1]
        edge_pos = np.where(dm ==1)
        edges_list.extend([(i+ n,j) for (i,j) in zip(*edge_pos)])
        edge_features_list.extend([[33,17,3,3,17] for edge_tuple in zip(*edge_pos)])
        edges_list.extend([(j,i + n) for (i,j) in zip(*edge_pos)])
        edge_features_list.extend([[33,17,3,3,17] for edge_tuple in zip(*edge_pos)])
    return edges_list ,edge_features_list
def pocketEdge(mol,adj=None,n1 = None,n2= None):
    edges_list = []
    edge_features_list = []
    if len(mol.GetBonds()) > 0: # mol has bonds
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx() + n1
            j = bond.GetEndAtomIdx() + n1
            edge_feature = bond_to_feature_vector(bond)
            # add edges in both directions
            edges_list.append((i, j))
            edge_features_list.append(edge_feature)
            edges_list.append((j, i))
            edge_features_list.append(edge_feature)
    if adj== None :
        return edges_list ,edge_features_list
    # add virtual aromatic nodes feature
    # add pocket
    else:
        n = len(mol.GetAtom()) + n1
        all_n = len(adj)
        adj -= np.eye(all_n)
        dm = adj[n:,n1:]
        edge_pos = np.where(dm ==1)
        edges_list.extend([(i+ n,j + n1) for (i,j) in zip(*edge_pos)])
        edge_features_list.extend([[33,17,3,3,17] for edge_tuple in zip(*edge_pos)])
        edges_list.extend([(j + n1,i + n) for (i,j) in zip(*edge_pos)])
        edge_features_list.extend([[33,17,3,3,17] for edge_tuple in zip(*edge_pos)])
    return edges_list ,edge_features_list



def getEdge(mols,n1,n2,adj = None):
    num_bond_features = 5
    mol,pocket = mols
    mol1_edge_idxs,mol1_edge_attr = molEdge(mol,adj,n1,n2)
    mol2_edge_idxs,mol2_edge_attr = pocketEdge(pocket,adj,n1,n2)
    edges_list = mol1_edge_idxs + mol2_edge_idxs
    edge_features_list = mol1_edge_attr + mol2_edge_attr

    if adj== None:
        pass
    else:
        #加入虚拟边，然后为虚拟边加入特征向量
        dm = adj[:n1,n1:n2]
        edge_pos = np.where(dm == 1)
        edges_list.extend([(i,j+n1) for (i,j) in zip(*edge_pos)])
        edge_features_list.extend([[32,16,2,2,16] for edge_tuple in zip(*edge_pos)])
        edges_list.extend([(j+n1,i) for (i,j) in zip(*edge_pos)])
        edge_features_list.extend([[32,16,2,2,16] for edge_tuple in zip(*edge_pos)])
    if len(edges_list) == 0:
        edge_index = np.empty((2, 0), dtype = np.int64)
        edge_attr = np.empty((0, num_bond_features), dtype = np.int64)
    else:
        edge_index = np.array(edges_list, dtype = np.int64).T
        edge_attr = torch.tensor(edge_features_list, dtype = torch.int64)

    return edge_index,edge_attr

In [9]:
np.empty((2, 0))

array([], shape=(2, 0), dtype=float64)

In [15]:
a

[(1, 2), (2, 1)]